In [1]:
import random
import numpy as np
import networkx as nx 
from dijkstar import Graph, find_path

In [2]:
def generate_instance_matrix(nb_vertex, min_weight, max_weight, density):
  graph = np.ones((nb_vertex, nb_vertex)) * np.inf
  for i in range(nb_vertex - 1):
    for j in range(i+1, nb_vertex):
      if random.uniform(0, 1) < density:
        weight = random.randint(min_weight, max_weight)
        graph[i, j] = weight
        graph[j, i] = weight
  for i,line in enumerate(graph):
    index_zeros = np.where(line == np.inf)[0]
    index_zeros = np.delete(index_zeros, np.where(index_zeros == i))
    for j in range(len(index_zeros) - (nb_vertex - 3)):
      index = random.choice(index_zeros)
      index_zeros = np.delete(index_zeros, np.where(index_zeros == index))
      weight = random.randint(min_weight, max_weight)
      graph[i, index] = weight
      graph[index, i] = weight
  return graph

def convert_graph(graph):
  graph_dijkstra = Graph()
  for i in range(len(graph)):
    for j in range(len(graph)):
      graph_dijkstra.add_edge(i, j,graph[i][j])
  return graph_dijkstra

def get_distance_between_nodes(graph, i, j):
  if (graph[i][j] != np.inf):
    return graph[i][j], [i, j]
  else:
    if i != j:
      shortest_path = find_path(graph, i,j)
      return shortest_path.total_cost, shortest_path.nodes
    else:
      return 0, None

def get_complete_graph(graph, node_list):
  all_paths = {}
  converted_graph = convert_graph(graph)
  complete_graph = np.zeros((len(node_list), len(node_list)))
  for i in node_list:
    voisins_array = np.delete(node_list, np.where(node_list == i))
    for j in voisins_array:
      distance, path = get_distance_between_nodes(converted_graph, i, j)
      converted_graph_i = np.where(node_list == i)[0][0]
      converted_graph_j = np.where(node_list == j)[0][0]
      complete_graph[converted_graph_i][converted_graph_j] = distance
      all_paths[(converted_graph_i, converted_graph_j)] = path
  return complete_graph, all_paths

def get_random_vertex(size, nb_vertex):
  random_vertex = []
  all_vertex = np.array(range(size))
  for _ in range(nb_vertex):
    value = random.choice(all_vertex)
    random_vertex.append(value)
    all_vertex = np.delete(all_vertex, np.where(all_vertex == value))
  return np.array(random_vertex)

def generate_interval():
  interval = random.choice(range(1,4))
  shift = random.choice(range(0, 11-interval))
  return(shift*60, (shift+interval)*60)

def generate_instance(nb_vertex_matrix, min_weight, max_weight, density, random_vertex):
  graph = generate_instance_matrix(nb_vertex_matrix, min_weight, max_weight, density)
  complete_graph, all_paths = get_complete_graph(graph, random_vertex)
  return graph, complete_graph, all_paths

def get_complete_path(path, all_paths):
  complete_path = []
  for i in range(len(path) - 1):
    complete_path.extend(all_paths[(path[i], path[i+1])][:-1])
    if i == len(path) - 2:
      complete_path.append(all_paths[(path[i], path[i+1])][-1])
  return complete_path

def is_path_in_interval(path, complete_graph, interval, vertex):
  weight = 0
  for i in range(np.where(np.array(path) == vertex)[0][0]):
    weight += complete_graph[path[i]][path[i+1]]
  if weight >= interval[0] and weight <= interval[1]:
    return True
  else:
    return False

def generate_neighbours(path, complete_graph, interval, vertex):
  neighbours = []
  for i in range(len(path)):
    for j in range(i + 1, len(path)):
      new_neighbour = path.copy()
      new_neighbour[i] = path[j]
      new_neighbour[j] = path[i]
      if is_path_in_interval(new_neighbour, complete_graph, interval, vertex):
        neighbours.append(new_neighbour)
  return neighbours

def generate_random_path(nb_vertex):
  path = np.arange(nb_vertex)
  np.random.shuffle(path)
  return path

In [3]:
# variables
nb_vertex_matrix = 100
min_weight = 10
max_weight = 60
density = 0.3
nb_vertex_chosen = 20

In [4]:
#random generation of a list of vertex among all the vertex available
random_vertex = get_random_vertex(nb_vertex_matrix, nb_vertex_chosen)

# generation of the graph, the complete graph and the list of all paths
graph, complete_graph, all_paths =\
generate_instance(
  nb_vertex_matrix = nb_vertex_matrix,
  min_weight = min_weight,
  max_weight = max_weight,
  random_vertex = random_vertex,
  density = density
)

In [5]:
#generate an interval instance
interval = generate_interval()

In [6]:
#generate a random path
path = generate_random_path(nb_vertex_chosen)

In [7]:
# generate the neighbours of a path
neighbours = generate_neighbours(path, complete_graph, interval, 18)

In [8]:
# print("graph")
# print(graph)
# print("complete_graph")
# print(complete_graph)
# print("all_paths")
# print(all_paths)
# print("random_vertex")
# print(random_vertex)
# print("interval")
# print(interval)

In [9]:
complete_path = get_complete_path(path, all_paths)

In [10]:
print(random_vertex)
print(path)
print(complete_path)


[18 54 66 95 36 99 10 76 16 46  1 29 51 94 89 30 53 34 96 27]
[ 4  9 13 14 18 17  5 11  0  7 16  1 15  6  2 19 12 10  8  3]
[36, 4, 14, 46, 92, 94, 92, 4, 89, 50, 96, 12, 19, 34, 45, 73, 99, 29, 74, 58, 18, 37, 41, 76, 56, 53, 23, 83, 54, 30, 16, 10, 66, 27, 43, 49, 61, 51, 61, 67, 1, 30, 16, 95]


In [15]:
from pulp import *
import numpy as np
X=[]
P = []
# variables du problème
for i in range(len(complete_graph)):
    for j in range(len(complete_graph)):
        X.append(LpVariable("X_" + str(i) + "_" + str(j), 0, 1, LpInteger))
        P.append(complete_graph[i][j])


# probleme
prob = LpProblem("plus court chemin", LpMinimize)

# objectif
# Somme de la multiplication des poids de chaque arc
prob += lpSum([P[i] * X[i] for i in range(len(P))])

# contraintes
# La somme d'un arc doit etre egale a 1
for i in range(len(complete_graph)):
    prob += lpSum([X[i]]) == 1

prob.solve()
print(LpStatus[prob.status])
print("Min=", value(prob.objective))

# variables resultat
for v in prob.variables():
    print("%s=%.2f"%(v.name,v.varValue), end=', ')


Optimal
Min= 777.0
X_0_0=1.00, X_0_1=1.00, X_0_10=1.00, X_0_11=1.00, X_0_12=1.00, X_0_13=1.00, X_0_14=1.00, X_0_15=1.00, X_0_16=1.00, X_0_17=1.00, X_0_18=1.00, X_0_19=1.00, X_0_2=1.00, X_0_3=1.00, X_0_4=1.00, X_0_5=1.00, X_0_6=1.00, X_0_7=1.00, X_0_8=1.00, X_0_9=1.00, X_10_0=0.00, X_10_1=0.00, X_10_11=0.00, X_10_12=0.00, X_10_13=0.00, X_10_14=0.00, X_10_15=0.00, X_10_16=0.00, X_10_17=0.00, X_10_18=0.00, X_10_19=0.00, X_10_2=0.00, X_10_3=0.00, X_10_4=0.00, X_10_5=0.00, X_10_6=0.00, X_10_7=0.00, X_10_8=0.00, X_10_9=0.00, X_11_0=0.00, X_11_1=0.00, X_11_10=0.00, X_11_12=0.00, X_11_13=0.00, X_11_14=0.00, X_11_15=0.00, X_11_16=0.00, X_11_17=0.00, X_11_18=0.00, X_11_19=0.00, X_11_2=0.00, X_11_3=0.00, X_11_4=0.00, X_11_5=0.00, X_11_6=0.00, X_11_7=0.00, X_11_8=0.00, X_11_9=0.00, X_12_0=0.00, X_12_1=0.00, X_12_10=0.00, X_12_11=0.00, X_12_13=0.00, X_12_14=0.00, X_12_15=0.00, X_12_16=0.00, X_12_17=0.00, X_12_18=0.00, X_12_19=0.00, X_12_2=0.00, X_12_3=0.00, X_12_4=0.00, X_12_5=0.00, X_12_6=0.00, X_